This file is for creating COCO-format traingin data set. The COCO format is required for FCOS source code version. 

In [ ]:
import h5py
import json
import numpy as np
import cv2 as cv
from pathlib import Path
from tqdm import tqdm

In [ ]:
path = Path('/home/tam/') / 'git' / 'cvppp2017_dataset'

In [ ]:
truth = h5py.File(path / 'CVPPP2017_training_truth.h5')

In [ ]:
info = {
    "description": "CVPPP2017 Leaf Segmentation Challenge Dataset",
    "year": 2022,
}

In [ ]:
licenses = [
    {
        "url": "http://creativecommons.org/licenses/by-nc/2.0/",
        "id": 2,
        "name": "Attribution-NonCommercial License"
    }
]

In [ ]:
categories = [
    {
        "supercategory": "plant",
        "id": 1,
        "name": "leaf"},
]

Before is the code that will extract leaf from the ground truth info and complie in the COCO-format

In [ ]:
images = []
annotations = []
image_id = 1
annotation_id = 1
for plant_id in tqdm(truth['A1'].keys()):
    image_name = truth['A1'][plant_id]['label_filename'][()].decode('ascii')
    h, w= truth['A1'][plant_id]['label'][()].shape
    images.append({
        "license": 2,
        "file_name": image_name,
        "height": h,
        "width": w,
        "id": image_id
    })

    for leaf_id in range(1, np.unique(truth['A1'][plant_id]['label'][()]).max()+1):
        mask = (truth['A1'][plant_id]['label'][()] == leaf_id).astype(np.uint8)
        contours, hierarchy = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        annotations.append({
            "segmentation": contours[0].ravel().tolist(),
            "area": cv.contourArea(contours[0]),
            "iscrowd": 0,
            "image_id": image_id,
            "bbox": list(cv.boundingRect(contours[0])),
            "category_id": 1,
            "id": annotation_id
        })

        annotation_id += 1
    image_id += 1


In [ ]:
coco_info = {}
coco_info['info'] = info
coco_info['licenses'] = licenses
coco_info['images'] = images
coco_info['annotations'] = annotations
coco_info['categories'] = categories

In [ ]:
with open('coco_label.json', 'w') as fo:
    fo.write(json.dumps(coco_info))